# Build a Coding Agent Workshop

The main goal of this workshop is to understand how "Project Bootstrappers" like [Lovable](https://lovable.dev/) are implemented in practice.

## Prerequisites

To get started, all we need is to have Python installed on our computer and an [OpenAI](https://openai.com/) account (or an alternative provider).


Install the required libraries in our Python virtual environment:
```bash
pip install jupyter django uv openai toyaikit
```

For local development, add this before we run Jupyter notebook:
```bash
export OPENAI_API_KEY='YOUR_KEY'
```


## Simple Chatbot vs. Agentic AI

ChatGPT is actually an agent because it has extra functionalities like creating images, performing web search, etc. A plain chatbot only uses LLM to answer questions. An agent can have access to different tools.

In [ ]:
from openai import OpenAI

In [ ]:
openai_client = OpenAI()

In [ ]:
import random

# An agent can decide whether to invoke this function
def make_joke(name):
    jokes = [
        f"Why did {name} bring a pencil to the party? Because he wanted to draw some attention!",
        f"Did you hear about {name}'s bakery? Business is on a roll!",
        f"{name} walked into a library and asked for a burger. The librarian said, 'This is a library.' So {name} whispered, 'Can I get a burger?'",
        f"When {name} does push-ups, the Earth moves down.",
        f"{name} told a chemistry joke... but there was no reaction.",
    ]
    return random.choice(jokes)

print(make_joke("Quinn"))

We need a description of the above function following a certain schema to tell a LLM that we have this function:

In [ ]:
make_joke_description = {
    "type": "function",
    "name": "make_joke",
    "description": "Generates a random personalized joke using the provided name.",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name to insert into the joke, personalizing the output.",
            }
        },
        "required": ["name"],
        "additionalProperties": False,
    },
}

In [ ]:
# Describe the kind of behavior we expect from the system
system_prompt = "You can make funny and original jokes."
# Query from the user
user_prompt = "Tell me a joke about Quinn."

chat_messages = [
    {"role": "developer", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Send a request to the model
response = openai_client.responses.create(
    # Feel free to experiment with better models
    model='gpt-4o-mini',
    input=chat_messages,
    # Make the chatbot agentic by providing it with tools
    tools=[make_joke_description]
)

In [ ]:
response.output

For orchestrating, we use [toyaikit](https://github.com/alexeygrigorev/toyaikit): a convenient interface for interacting with OpenAI models without having to write a lot of code. 

Note that it is for education and not for use in production.

```bash
pip install -U toyaikit
```

Make sure our version is at least 0.0.3.

In [ ]:
from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.llm import OpenAIClient
from toyaikit.chat.runners import OpenAIResponsesRunner

tools_obj = Tools()
# Define the tools that we want to use below
tools_obj.add_tool(make_joke, make_joke_description)

# Define the chat interface
chat_interface = IPythonChatInterface()
openai_client = OpenAIClient(client=OpenAI())

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=system_prompt,
    chat_interface=chat_interface,
    llm_client=openai_client
)

In [ ]:
# Get a list of all the tools
tools_obj.get_tools()

Walkthrough:
- You: tell me a joke
- You: tell me a personalized joke
- You: [Name]
- You: stop

In [ ]:
runner.run()

## Coding Agent

Lovable works with predefined JavaScript templates, which it then makes changes to a bunch of files in order to have working results; it doesn't start from scratch. Here, we will implement our coding agent in a similar way but with Django templates. Feel free to experiment with templates of other frameworks (e.g., FastAPI, Flask).

First, create a simple Django template such that whatever we do will be based on this template. Later on, we'll copy this folder with the template to a new folder and let the agent work on this new folder.

To learn more about building with Django, visit: https://www.djangoproject.com/start/

```bash
mkdir django_template
cd django_template

uv init
# Remove 'main.py' as we do not need it
rm main.py

uv add django

# Create an empty project called myproject in the current directory
uv run django-admin startproject myproject .
# Create an empty app called myapp
uv run python manage.py startapp myapp
```

Add the new app (`myapp`) into `myproject/settings.py`'s `INSTALLED_APPS`:
```python
INSTALLED_APPS = [
    # ... other apps
    'myapp',
]
```

Create a `Makefile` with the following useful commands:

```makefile
.PHONY: install migrate run

install:
	uv sync --dev

migrate:
	uv run python manage.py migrate

run:
	uv run python manage.py runserver
```

Create the base html template in `templates/base.html` under the project root directory (`/django_template`):
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}{% endblock %}</title>
</head>
<body>
    {% block content %}{% endblock %}
</body>
</html>
```
Django will generate all HTML pages based on this base template. 

Now, add this `templates` directory to the `settings.py` file:
```python
TEMPLATES = [{
    # Add here
    'DIRS': [BASE_DIR / 'templates'],
    ...
}
```

Then, add the home view for our app:
```python
# myapp/views.py
def home(request):
    return render(request, 'home.html')

# myproject/urls.py
from myapp import views

urlpatterns = [
    # ...
    # Add the new view below
    path('', views.home, name='home'),
]
```

Create a template for home in `templates/home.html` under `/myapp`

```html
<!-- myapp/templates/home.html -->
{% extends 'base.html' %}

{% block content %}
<h1>Home</h1>
{% endblock %}
```

Finally, add some styling with TailwindCSS and FontAwesome to our `base.html` template:
```html
<!-- django_template/templates/base.html -->
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>{% block title %}My Site{% endblock %}</title>
  <!-- Add TailwindCSS -->
  <script src="https://cdn.tailwindcss.com"></script>
  <!-- Add FontAwesome -->
  <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.1/css/all.min.css" rel="stylesheet">
  {% load static %}
  <link href="{% static 'css/styles.css' %}" rel="stylesheet">
  {% block extra_head %}{% endblock %}
</head>

<body class="min-h-screen flex flex-col">
  <header class="bg-gray-800 text-white">
    <nav class="container mx-auto px-4 py-3">
      <div class="flex items-center justify-between">
        <a href="/" class="text-xl font-bold">My Site</a>
        <div class="space-x-4">
          <a href="/" class="hover:text-gray-300">Home</a>
          <a href="/about" class="hover:text-gray-300">About</a>
          <a href="/contact" class="hover:text-gray-300">Contact</a>
        </div>
      </div>
    </nav>
  </header>

  <main class="container mx-auto px-4 py-8 flex-grow">
    {% if messages %}
    <div class="messages space-y-2">
      {% for message in messages %}
      <div class="message {{ message.tags }} p-4 rounded-lg {% if message.tags == 'success' %}bg-green-100 text-green-700{% elif message.tags == 'error' %}bg-red-100 text-red-700{% else %}bg-blue-100 text-blue-700{% endif %}">
        {{ message }}
      </div>
      {% endfor %}
    </div>
    {% endif %}

    {% block content %}
    {% endblock %}
  </main>

  <footer class="bg-gray-800 text-white py-6">
    <div class="container mx-auto px-4 text-center">
      <p>&copy; {% now "Y" %} My Site. All rights reserved.</p>
    </div>
  </footer>

  {% block extra_body %}{% endblock %}
</body>

</html>
```

Alternatively, we can download a prebuilt Django template:
```bash
git clone https://github.com/alexeygrigorev/django_template.git

cd django_template
# Install dependencies
uv sync

# Activate the database
make migrate
# Run the app
make run
```


Before we start implementing our coding agent, we need a function to copy the Django template into a separate folder:

In [ ]:
import os
import shutil

def start():
    project_name = input("Enter the new Django project name: ").strip()
    if not project_name:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_name):
        print(f"Directory '{project_name}' already exists. Please choose a different name or remove the existing directory.")
        return

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return project_name

And, this is how we use it:

Walkthrough:
- Enter the new Django project name:  `todo`
- Django template copied to 'todo' directory.

In [ ]:
project_name = start()

Next, we need to define a few functions for the coding agent:
- Read file
- Write file
- Execute a bash command
- Potentially also: list files and grep

We will use an AI assistant like ChatGPT, Claude, or Cursor for creating these files.

When debugging the code, we can use this magic command for automatic file reload:
```python
%load_ext autoreload
%autoreload 2
```

The final resulting code can be seen in `tools.py`

```python
# tools.py
"""
Tool functions and their descriptions for the coding agent.
"""

from pathlib import Path
import os
import subprocess


class AgentTools:
    # Directories to skip when building file tree
    SKIP_DIRS = {
        ".venv",
        "__pycache__",
        ".git",
        ".pytest_cache",
        ".mypy_cache",
        ".coverage",
        "node_modules",
        ".DS_Store",
    }

    def __init__(self, project_dir: Path):
        """
        Initialize AgentTools with the given project directory.

        Parameters:
            project_dir (Path): The root directory of the project.
        """
        self.project_dir = project_dir

    def read_file(self, filepath: str) -> None:
        """
        Read and return the contents of a file at the given relative filepath.

        Parameters:
            filepath (str): Path to the file, relative to the project directory.
        Returns:
            str: Contents of the file.
        """
        abs_path = self.project_dir / filepath
        with open(abs_path, "r", encoding="utf-8") as f:
            return f.read()

    def write_file(self, filepath: str, content: str) -> None:
        """
        Write the given content to a file at the given relative filepath, creating directories as needed.

        Parameters:
            filepath (str): Path to the file, relative to the project directory.
            content (str): Content to write to the file.
        Returns:
            None
        """
        abs_path = self.project_dir / filepath
        abs_path.parent.mkdir(parents=True, exist_ok=True)
        with open(abs_path, "w", encoding="utf-8") as f:
            f.write(content)

    def see_file_tree(self, root_dir: str = ".") -> list[str]:
        """
        Return a list of all files and directories under the given root directory,
        relative to the project directory.

        Parameters:
            root_dir (str): Root directory to list from, relative to the project directory.
                           Defaults to ".".
        Returns:
            list[str]: List of relative paths for all files and directories.
        """
        abs_root = self.project_dir / root_dir
        tree = []

        for dirpath, dirnames, filenames in os.walk(abs_root):
            # Remove blacklisted directories from dirnames to prevent os.walk from entering them
            for skip_dir in list(dirnames):
                if skip_dir in self.SKIP_DIRS:
                    dirnames.remove(skip_dir)

            for name in dirnames + filenames:
                full_path = os.path.join(dirpath, name)
                rel_path = os.path.relpath(full_path, self.project_dir)
                tree.append(rel_path)

        return tree

    def execute_bash_command(
        self, command: str, cwd: str = None
    ) -> tuple[str, str, int]:
        """
        Execute a bash command in the shell and return its output, error, and exit code. Blocks running the Django development server (runserver).

        Parameters:
            command (str): The bash command to execute.
            cwd (str, optional): Working directory to run the command in, relative to the project directory. Defaults to None.
        Returns:
            tuple: (stdout (str), stderr (str), returncode (int))
        """
        # Block running the Django development server
        if "runserver" in command:
            return (
                "",
                "Error: Running the Django development server (runserver) is not allowed through this tool.",
                1,
            )

        abs_cwd = (self.project_dir / cwd) if cwd else self.project_dir

        result = subprocess.run(
            command,
            shell=True,
            capture_output=True,
            text=True,
            cwd=abs_cwd,
            timeout=15,
            encoding='utf-8',
            errors='replace',
        )

        return result.stdout, result.stderr, result.returncode

    def search_in_files(
        self, pattern: str, root_dir: str = "."
    ) -> list[tuple[str, int, str]]:
        """
        Search for a pattern in all files under the given root directory and return a list of matches as (relative path, line number, line content).

        Parameters:
            pattern (str): Pattern to search for in files.
            root_dir (str): Root directory to search from, relative to the project directory. Defaults to ".".
        Returns:
            list[tuple]: List of (relative path, line number, line content) for each match.
        """
        abs_root = self.project_dir / root_dir
        matches = []
        for dirpath, _, filenames in os.walk(abs_root):
            for filename in filenames:
                filepath = os.path.join(dirpath, filename)
                try:
                    with open(filepath, "r", encoding="utf-8") as f:
                        for i, line in enumerate(f, 1):
                            if pattern in line:
                                rel_path = os.path.relpath(filepath, self.project_dir)
                                matches.append((rel_path, i, line.strip()))
                except Exception:
                    continue
        return matches

```

**Note:** For bash, you want to disable running "runserver". If we allow the agent to run it in Jupyter notebook, it will hang up the environment.

So if we put the code inside `AgentTools` class, we can use it like that:

In [ ]:
import tools
from pathlib import Path

coding_agent_tools = tools.AgentTools(Path(project_name))

In [ ]:
# coding_agent_tools.see_file_tree()

In [ ]:
# coding_agent_tools.read_file('templates/base.html')

In [ ]:
# coding_agent_tools.search_in_files('html')

In [ ]:
# coding_agent_tools.execute_bash_command('echo 123')

In [ ]:
tools_obj = Tools()
tools_obj.add_tools(coding_agent_tools)

In [ ]:
tools_obj.get_tools()

Start with a simple prompt:

```python
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided
Django project template according to user instructions.
"""
```

Then, slowly make refinements (with the help of ChatGPT) such that eventually it becomes more sophisticated like the prompt below:

In [ ]:
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project's structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree

├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runproject", but you can execute other commands to check if the project is working.
- Make sure you use TailwindCSS styles for making the result look beautiful
- Keep the original URL for TailwindCSS
- Use pictograms and emojis when possible. Font-awesome is available
- Avoid putting complex logic to templates - do it on the server side when possible
"""

Iterate over the prompt until we get a result we're satisfied with; the more precise and complete the system prompt we give to the agent, the better the outcome. The current version of the prompt is not the best and can be further improved. 

Generally, we would spend most of the time writing the system prompt (also known as prompt engineering) when working with agents.

In [ ]:
chat_interface = IPythonChatInterface()
openai_client = OpenAIClient(client=OpenAI())

runner = OpenAIResponsesRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=chat_interface,
    llm_client=openai_client
)

Now, give the agent a go by telling it the kind of app or use case we want to implement:

Walkthrough:
- You: todo list
- You: make the forms more beautiful and delete doesn't work
- You: stop

In [ ]:
runner.run()

Once our coding agent completes the task, go to the project directory, and run the app:
```bash
make run
```

If the app doesn't work, continue the conversation with the agent until it's fixed.

## OpenAI Agents SDK

Earlier in our code implementation, we used `toyaikit`, which is not a production-ready library. In production, we use other advanced libraries like [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/). 

```bash
pip install openai-agents
```

In [ ]:
from agents import Agent, function_tool

To define tools, we use the `@function_tool` annotation or decorator:

In [ ]:
import random

@function_tool
def make_joke(name: str) -> str:
    """
    Generates a personalized joke using the provided name.

    Parameters:
        name (str): The name to insert into the joke.

    Returns:
        str: A joke with the name included.
    """
    jokes = [
        f"Why did {name} bring a pencil to the party? Because he wanted to draw some attention!",
        f"Did you hear about {name}'s bakery? Business is on a roll!",
        f"{name} walked into a library and asked for a burger. The librarian said, 'This is a library.' So {name} whispered, 'Can I get a burger?'",
        f"When {name} does push-ups, the Earth moves down.",
        f"{name} told a chemistry joke... but there was no reaction.",
    ]
    return random.choice(jokes)

Note that we also have to include type hints and docstrings.

If it's a function that is already defined somewhere, then we can do this without using the `@function_tool` annotation on our `make_joke` function:
```python
make_joke_tool = function_tool(make_joke)
```

Now, define an agent:

In [ ]:
joke_system_prompt = """
You can make funny and original jokes.
Find out the user's name to make the joke personalized.
"""

joke_agent = Agent(
    name="JokeAgent",
    instructions=joke_system_prompt,
    tools=[make_joke],
    # tools=[make_joke_tool],
    model='gpt-4o-mini'
)

`toyaikit` has a runner for it, so we don't need to write boilerplate code.

Walkthrough:
- You: tell me a joke
- You: [Name]
- You: stop

In [ ]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

interface = IPythonChatInterface()
runner = OpenAIAgentsSDKRunner(
    chat_interface=interface,
    agent=coding_agent
)

# OpenAI Agents SDK is async, so we need to use await here
await runner.run()

[Refer to the code for the runner](https://github.com/alexeygrigorev/toyaikit/blob/main/toyaikit/chat/runners.py#L75) to see how it's implemented (useful if we want to work with the Agents SDK in the future).

Let's extend it to our coding agent example. First, create the `AgentTools` class:

In [ ]:
agent_tools = tools.AgentTools(Path(project_name))

The `wrap_instance_methods` is a helper utility that we can use to automatically scan all methods of an object and apply the decorator:

In [ ]:
from toyaikit.tools import wrap_instance_methods

coding_agent_tools_list = wrap_instance_methods(function_tool, agent_tools)

Without it, we would need to wrap the tools with `function_tool` to obtain the same result:
```python
coding_agent_tools_list = [
    function_tool(agent_tools.execute_bash_command),
    function_tool(agent_tools.read_file),
    function_tool(agent_tools.search_in_files),
    function_tool(agent_tools.see_file_tree),
    function_tool(agent_tools.write_file)
]
```

In [ ]:
coding_agent_tools_list

Create the coding agent:

In [ ]:
coding_agent = Agent(
    name="CodingAgent",
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list,
    model='gpt-4o-mini'
)

And run it with the same walkthrough flow:

In [ ]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=interface,
    agent=coding_agent
)

await runner.run()

## Pydantic AI

The next library we will explore is [Pydantic AI](https://ai.pydantic.dev/): A Python agent framework designed to help us quickly, confidently, and painlessly build production grade applications and workflows with Generative AI.

We will jump directly into implementing the coding agent. Feel free to check out the docs to see how to build a simpler agent with the `make_joke` function.

```
pip install pydantic-ai
```

In [ ]:
from pydantic_ai import Agent
from toyaikit.tools import get_instance_methods

Let's get our coding agent to build a flashcard app like Anki.

Walkthrough:
- Enter the new Django project name:  `anki`
- Django template copied to 'anki' directory.

In [ ]:
project_name = start()

Next, define the tools by simply going through all the methods of this `agent_tools` object:

In [ ]:
agent_tools = tools.AgentTools(Path(project_name))
coding_agent_tools_list = get_instance_methods(agent_tools)

Without `get_instance_methods` helper utility, this is what we would need to do:
```python
coding_agent_tools_list = [
    agent_tools.execute_bash_command,
    agent_tools.read_file,
    agent_tools.search_in_files,
    agent_tools.see_file_tree,
    agent_tools.write_file
]
```

In [ ]:
coding_agent_tools_list

Next, define the Agent:

In [ ]:
coding_agent = Agent(
    'anthropic:claude-3-5-sonnet-latest',
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list
)

And run it:

Walkthrough:
- You: anki app
- You: stop

In [ ]:
from toyaikit.chat.runners import PydanticAIRunner

runner = PydanticAIRunner(
    chat_interface=chat_interface,
    agent=coding_agent
)

await runner.run()

Take some time to learn [how the run() function is implemented](https://github.com/alexeygrigorev/toyaikit/blob/main/toyaikit/chat/runners.py#L129) in `toyaikit`.


With Pydantic AI, it's easy to switch to another LLM provider:
```python
coding_agent = Agent(
    'openai:gpt-4o-mini',
    instructions=DEVELOPER_PROMPT,
    tools=coding_agent_tools_list
)
```

## Z.ai

Finally, let's try a different LLM provider: [Z.ai](https://docs.z.ai/guides/overview/quick-start). 

Z.ai, like most LLM providers, follow OpenAI API, so we can simply use the OpenAI client:

In [ ]:
from openai import OpenAI

zai_client = OpenAI(
    api_key=os.getenv('ZAI_API_KEY'),
    base_url='https://api.z.ai/api/paas/v4/'
)

This version doesn't support the `responses` API like OpenAI's latest models, so we will need to use the older `chat.completions` API.

Here we will use:
- `OpenAIChatCompletionsRunner` instead of `OpenAIResponsesRunner`
- `OpenAIChatCompletionsClient` insead of `OpenAIClient`

The rest is similar, including tools definition:

In [ ]:
from toyaikit.tools import Tools

from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIChatCompletionsRunner
from toyaikit.llm import OpenAIChatCompletionsClient

In [ ]:
agent_tools = tools.AgentTools(Path(project_name))

tools_obj = Tools()
tools_obj.add_tools(agent_tools)

# glm-4.5 is a reasoning model
llm_client = OpenAIChatCompletionsClient(model='glm-4.5', client=zai_client)
chat_interface = IPythonChatInterface()

runner = OpenAIChatCompletionsRunner(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=chat_interface,
    llm_client=llm_client
)

In [ ]:
runner.run()